&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&ensp;
[Home Page](../START_HERE.ipynb)

[Previous Notebook](02-SGD.ipynb)
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
[1](01-LinearRegression-Hyperparam.ipynb)
[2](02-SGD.ipynb)
[3]
[4](04_CuML_Solution.ipynb)
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
[Next Notebook](04_CuML_Solution.ipynb)

# CuML- Exercise
Scikit-Learn is an incredibly powerful toolkit that allows data scientists to quickly build models from their data, and it one of the most common and useful tools in the Python data science ecosystem. cuML is the RAPIDS library that implements similar machine learning algorithms that use CUDA to run on GPUs, with an API that mirrors the Scikit-learn one as much as possible.

In this notebook we present a small exercise for new users to experiment with CuML and apply their knowledge on a real world machine learning dataset. We will be working on the Car Accidents dataset that we started preprocessing in the CuDF tutorial. This is a countrywide car accident dataset, which covers 49 states of the USA. The accident data are collected from February 2016 to June 2020, using two APIs that provide streaming traffic incident (or event) data. These APIs broadcast traffic data captured by a variety of entities, such as the US and state departments of transportation, law enforcement agencies, traffic cameras, and traffic sensors within the road-networks. Currently, there are about 3.5 million accident records in this dataset. If you skipped that tutorial, you can download the processed dataset here.

# Challenge

We begin by perfoming some data manipulation using Scikit learn preprocessing and removing any class imbalance. The actual exercise begins <a href= '#exercise'> here</a>, where we have provided the implementation of 3 different Scikit-learn models and you have to convert them to CuML and evaluate the performance difference.

The first step is downloading the dataset and putting it in the data directory, for using in this tutorial. Download the dataset here, and place it in (host/data) folder. Now we will import the necessary libraries.

In [5]:
import matplotlib.pyplot as plt
import numpy as np; print('NumPy Version:', np.__version__)
%matplotlib inline
import sys
import sklearn; print('Scikit-Learn Version:', sklearn.__version__)
from sklearn.linear_model import LinearRegression


from sklearn import preprocessing 
import pandas as pd
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_curve, auc
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
import cudf
import cupy

# import for visualization
import matplotlib.pyplot as plt

# import for model building
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from cuml.linear_model import MBSGDRegressor as cumlSGD
from sklearn.linear_model import SGDRegressor as skSGD
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error

from cuml.ensemble import RandomForestClassifier as curfc
from sklearn.ensemble import RandomForestClassifier as skrfc

from cuml import make_regression
from cuml.linear_model import LinearRegression as cuLinearRegression
from cuml.metrics.regression import r2_score
from sklearn.linear_model import LinearRegression as skLinearRegression

from cuml.neighbors import KNeighborsClassifier as KNeighborsC
from sklearn.neighbors import KNeighborsClassifier
from cuml.linear_model import MBSGDClassifier as cumlMBSGDClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from cuml import Ridge
from cuml.linear_model import Ridge
from sklearn.linear_model import Ridge
from cuml import LogisticRegression
from sklearn.linear_model import LogisticRegression as skLogistic
from cuml.linear_model import ElasticNet
from sklearn import linear_model

from cuml.linear_model import Lasso
from cuml.solvers import SGD as cumlSGD

NumPy Version: 1.19.2
Scikit-Learn Version: 0.23.1


Let's read the dataframe from the csv which was processed in the previous tutorial and stored in the data folder.

In [7]:
%time 
df = pd.read_csv('../data/data_proc.csv')
print(df)

CPU times: user 1e+03 ns, sys: 1 µs, total: 2 µs
Wall time: 5.96 µs


FileNotFoundError: [Errno 2] No such file or directory: '../data/data_proc.csv'

Drop the unnecessary columns which got added while reading the file.

In [3]:
df = df.drop(columns = ["Unnamed: 0"])

NameError: name 'df' is not defined

Observe the dataset by printing the first 5 rows using the head function.

In [4]:
df.head()

NameError: name 'df' is not defined

Drop any null values that may be present.

In [5]:
df = df.dropna()

We are continuing a bit of the preprocessing that is easier using Scikit-learn and can use Label encoding to convert the labels to numbers without increasing the dimensions of our dataset. Label encoder converts the string categorical values to numbers.
Eg. [Chicago, New York, Mumbai] would get encoded to [0, 1, 2].

In [6]:
%%time
link to label encoder
label_encoder = preprocessing.LabelEncoder() 
df['County']= label_encoder.fit_transform(df['County']) 
df['State']= label_encoder.fit_transform(df['State'])
df['Weather_Condition']= label_encoder.fit_transform(df['Weather_Condition'])

df['Source'] = label_encoder.fit_transform(df['Source'])

df['Sunrise_Sunset'] = label_encoder.fit_transform(df['Sunrise_Sunset'])
df['Civil_Twilight'] = label_encoder.fit_transform(df['Civil_Twilight'])
df['Nautical_Twilight'] = label_encoder.fit_transform(df['Nautical_Twilight'])
df['Astronomical_Twilight'] = label_encoder.fit_transform(df['Astronomical_Twilight'])

df['Amenity'] = label_encoder.fit_transform(df['Amenity'])
df['Bump'] =label_encoder.fit_transform(df['Bump'])
df['Crossing'] = label_encoder.fit_transform(df['Crossing'])
df['Give_Way'] = label_encoder.fit_transform(df['Give_Way'])
df['Junction'] =label_encoder.fit_transform(df['Junction'])
df['No_Exit'] = label_encoder.fit_transform(df['No_Exit'])
df['Railway'] = label_encoder.fit_transform(df['Railway'])
df['Roundabout'] = label_encoder.fit_transform(df['Roundabout'])

df['Station'] = label_encoder.fit_transform(df['Station'])
df['Stop'] = label_encoder.fit_transform(df['Stop'])
df['Traffic_Calming'] = label_encoder.fit_transform(df['Traffic_Calming'])
df['Traffic_Signal'] = label_encoder.fit_transform(df['Traffic_Signal'])
df['Turning_Loop'] =label_encoder.fit_transform(df['Turning_Loop'])

CPU times: user 14.6 ms, sys: 515 µs, total: 15.1 ms
Wall time: 14.9 ms


Let's continue with exploring the dataset. We can check how the values are distributed in different categories.

In [ ]:
df['Severity'].value_counts()

The distribution across all the severities is imbalanced and Machine Learning algorithms tend to produce unsatisfactory classifiers when faced with imbalanced datasets.So we will convert this dataset to the necessary form by performing class balancing using up sampling. Up-sampling is the process of randomly duplicating observations from the minority class in order to reinforce its signal.

- First, we'll separate observations from each class into different DataFrames.
- Next, we'll resample the minority class with replacement, setting the number of samples to match that of the majority class.
- Finally, we'll combine the up-sampled minority class DataFrame with the original majority class DataFrame.

In [7]:
%%time
# Class Balancing | Using Up Sampling

# Separate majority and minority classes
df_s1 = df[df['Severity']==1]
df_s2 = df[df['Severity']==2]
df_s3 = df[df['Severity']==3]
df_s4 = df[df['Severity']==4]

count = max(df_s1.count()[0], df_s2.count()[0], df_s3.count()[0], df_s4.count()[0])

# Upsample minority class
df_s1 = resample(df_s1, replace=df_s1.count()[0]<count, n_samples=count, random_state=42)
df_s2 = resample(df_s2, replace=df_s2.count()[0]<count, n_samples=count, random_state=42)
df_s3 = resample(df_s3, replace=df_s3.count()[0]<count, n_samples=count, random_state=42)
df_s4 = resample(df_s4, replace=df_s4.count()[0]<count, n_samples=count, random_state=42)
 
# Combine majority class with upsampled minority class
df = pd.concat([df_s1, df_s2, df_s3, df_s4])
 
# Display new class counts
df.groupby(by='Severity')['Severity'].count()

CPU times: user 30.1 ms, sys: 4.45 ms, total: 34.6 ms
Wall time: 34.1 ms


Severity
1    10584
2    10584
3    10584
4    10584
Name: Severity, dtype: int64

Now we will separate our target data column from the other columns and encode categorical features present in the dataframe as an integer array.

In [8]:
%%time
# Set the target for the prediction
target='Severity' 
cols = df.select_dtypes(include='object').columns

# set X and y
y = df[target]
X = df.drop(target, axis=1)

# Create the encoder.
encoder = OrdinalEncoder()
X[cols] = encoder.fit_transform(X[cols])



CPU times: user 7.11 ms, sys: 5.16 ms, total: 12.3 ms
Wall time: 11.5 ms


Now we will use the train test split function of scikit learn to create the train and test datasets. The train-test split is a technique for evaluating the performance of a machine learning algorithm. The procedure involves taking a dataset and dividing it into two subsets. The first subset is used to fit the model and is referred to as the training dataset. The second subset is not used to train the model; instead, the input element of the dataset is provided to the model, then predictions are made and compared to the expected values. This second dataset is referred to as the test dataset.

- Train Dataset: Used to fit the machine learning model.
- Test Dataset: Used to evaluate the fit machine learning model.


The objective is to estimate the performance of the machine learning model on new data: data not used to train the model.

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size = 0.3, random_state=0)

Now the data is in the required format and ready to be fed to our model. We will now convert the dataframe to a CuDF dataframe. 

In [1]:
#Convert the data to CuDF dataframes here
%%time
X_cudf_train = cudf.DataFrame.from_pandas(X_train)
X_cudf_test = cudf.DataFrame.from_pandas(X_test)

y_cudf_train = cudf.Series(y_train.values)
y_cudf_test = cudf.Series(y_test.values)


<a id= 'exercise'></a>

#### Your exercise begins here. Provided below are 3 ML models in Scikit-learn, which you have to convert to CuML and evaluate the performance difference.

# Logistic Regression

Logistic regression is a statistical model that in its basic form uses a logistic function to model a binary dependent variable.

## Scikit-learn

### Fit

In [19]:
%%time
clf = skLogistic()
clf.fit(X_train, y_train)


CPU times: user 20 s, sys: 50.9 s, total: 1min 10s
Wall time: 1.9 s


/opt/conda/envs/rapids/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

### Evaluate

In [20]:
%%time
print(clf.score(X_test, y_test))

0.501299110306275
CPU times: user 81.7 ms, sys: 193 ms, total: 275 ms
Wall time: 7.28 ms


# Nearest Neighbours Classifier

NearestNeighbors implements unsupervised nearest neighbors learning. It acts as a uniform interface to three different nearest neighbors algorithms: BallTree, KDTree, and a brute-force algorithm based on routines in sklearn.metrics.pairwise. The choice of neighbors search algorithm is controlled through the keyword 'algorithm', which must be one of ['auto', 'ball_tree', 'kd_tree', 'brute']. When the default value 'auto' is passed, the algorithm attempts to determine the best approach from the training data. 

## Scikit-learn

### Fit

In [31]:
%%time
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)

CPU times: user 522 ms, sys: 4.43 ms, total: 527 ms
Wall time: 526 ms


KNeighborsClassifier(n_neighbors=3)

### Evaluate

In [32]:
%%time
print(neigh.score(X_test, y_test))

0.8876466419966932
CPU times: user 1.15 s, sys: 5.22 ms, total: 1.15 s
Wall time: 1.15 s




## ElasticNet Classifier

Elastic Net first emerged as a result of critique on lasso, whose variable selection can be too dependent on data and thus unstable. The solution is to combine the penalties of ridge regression and lasso to get the best of both worlds. Ridge Regression, which penalizes sum of squared coefficients (L2 penalty). Lasso Regression, which penalizes the sum of absolute values of the coefficients (L1 penalty).

### Scikit-learn model

#### Fit

In [11]:
%%time
regr = ElasticNet()
regr.fit(X_train, y_train)

CPU times: user 163 ms, sys: 62.1 ms, total: 225 ms
Wall time: 226 ms


ElasticNet(alpha=1.0, l1_ratio=0.5, fit_intercept=True, normalize=False, max_iter=1000, tol=0.001, selection='cyclic', handle=<cuml.raft.common.handle.Handle object at 0x7fd97c163210>, output_type='numpy', verbose=4)

#### Evaluate

In [12]:
%%time
X_test = X_test.astype(np.float64)
y_test = y_test.astype(np.float64)
print(regr.score(X_test,y_test))

0.22519596677633613
CPU times: user 5.97 ms, sys: 2.98 ms, total: 8.96 ms
Wall time: 8.11 ms


# CONCLUSION

Let's compare the performance of our solution!

| Algorithm     | Implementation | Accuracy      | Time |
| ----------- | ----------- | ----------- | ----------- | 
| Logistic Regression     | Scikit-learn        | 0.5     | 1.9 s       |  
| Nearest Neighbours Classifier     | Scikit-learn       | 0.88      | 526 ms       |
| ElasticNet    | Scikit-learn       | 0.225      | 226 ms      | 

Here are some reasons why Nearest Neighbours could be working so well for our dataset:
- Flexible to feature/distance choices
- Naturally handles multi-class cases
- Can do well in practice with enough representative data

Wow! This was an interesting exercise. We hope you enjoyed applying your machine learning skills and appreciated the GPU boost provided by RAPIDS. Use the Work Book  [here](Workbook.ipynb) to complete the exercise and code while obtaining your optimal solution, and refer to the hints provided in it. If you were unable to solve this, check out our sample solution in the next lab, but don't restrict yourself to the algorithms we have used. CuML supports many ML models which can provide interesting results on this dataset.

# References

- Moosavi, Sobhan, Mohammad Hossein Samavatian, Srinivasan Parthasarathy, and Rajiv Ramnath. “A Countrywide Traffic Accident Dataset.”, 2019.

- Moosavi, Sobhan, Mohammad Hossein Samavatian, Srinivasan Parthasarathy, Radu Teodorescu, and Rajiv Ramnath. "Accident Risk Prediction based on Heterogeneous Sparse Data: New Dataset and Insights." In proceedings of the 27th ACM SIGSPATIAL International Conference on Advances in Geographic Information Systems, ACM, 2019.

- If you need to refer to the dataset, you can download it [here](https://www.kaggle.com/sobhanmoosavi/us-accidents).

<center><a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a></center><br />This dataset is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

## Licensing
  
This material is released by NVIDIA Corporation under the Creative Commons Attribution 4.0 International (CC BY 4.0).

[Previous Notebook](02-SGD.ipynb)
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
[1](01-LinearRegression-Hyperparam.ipynb)
[2](02-SGD.ipynb)
[3]
[4](04_CuML_Solution.ipynb)
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
[Next Notebook](04_CuML_Solution.ipynb)

&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&ensp;
[Home Page](../START_HERE.ipynb)